In [5]:
from qiskit import *
import subprocess
from qiskit.tools.monitor import job_monitor
import numpy as np



In [ ]:
IBMQ.load_account()

In [49]:
print(IBMQ.providers())
provider = IBMQ.get_provider(group='open')
provider.backends()
backend = provider.get_backend('ibmq_16_melbourne')
sim_backend=provider.get_backend('ibmq_qasm_simulator')
total_qubits=backend.configuration().num_qubits

[<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>]


In [7]:
import  math
from qiskit.tools.visualization import plot_histogram


def showResults(counts, nbit, bitArr):

    res_map = getResMap(counts, nbit, bitArr)
    plot_histogram(res_map)




def getResMap(counts, nbit, bitArr):
    res_map = {}
    totalSize = 1 << nbit;
    # print("total map size:", totalSize)
    for i in range(0, totalSize):
        str = ""
        for j in range(0, nbit):
            if (((i >> j) & 1) == 0):
                str = '0' + str
            else:
                str = '1' + str
        res_map[str] = 0

    for str in counts:
        m = str[::-1]
        s = ""
        for i in bitArr:
            s = s + m[i]

        res_map[s] = res_map[s] + counts[str]

    return res_map


def getPerformanceMeasure(counts, nbit, bitArr, expectedPattern):  # assumption : only single expected pattern. i.e
    res_map = getResMap(counts, nbit, bitArr)  # no superpositional output

    secondBest = 0
    for s in res_map:
        if (s != expectedPattern and res_map[s] > secondBest):
            secondBest = res_map[s]
    # if (res_map[expectedPattern] < secondBest):
    #     return 0
    # return res_map[expectedPattern]-secondBest
    return res_map[expectedPattern],secondBest;

def getExpectedPattern(counts, nbit, bitArr):  # assumption : only single expected pattern. i.e
    res_map = getResMap(counts, nbit, bitArr)  # no superpositional output

    BestVal = -1;
    BestPattern=''
    for s in res_map:
        if ( res_map[s] > BestVal):
            BestVal = res_map[s]
            BestPattern = s ;
    # if (res_map[expectedPattern] < secondBest):
    #     return 0
    # return res_map[expectedPattern]-secondBest
    return BestPattern;





In [8]:
def run_sub_process(ckt_name,p_id,config):
    p = subprocess.run('mapper_'+str(p_id)+'m.exe '+config+' '+str(ckt_name)+' out_'+str(p_id)
                       +'.qasm layout_'+str(p_id)+'.txt')
    # print(p)
    return p.returncode

def get_a_maped_qasm(ckt_name,p_id,config):
    # print(qasm)
    # in_qasm=open('input'+str(ckt_num)+'.qasm','w')
    # in_qasm.write(qasm)
    # in_qasm.close()

    while(run_sub_process(ckt_name,p_id,config)==1):
        print(p_id,config)
        print("Try runing again")

    # out_qasm=open('out.qasm','r')

    # qasm=""
    # for line in out_qasm:
    #     qasm+=line
    #
    # out_qasm.close()
    layout={}
    time=0.0
    for line in open('layout_'+p_id+'.txt','r'):
        splits=line.split()
        if(len(splits)>1):
            q1,q2=splits
            q1=int(q1)
            q2=int(q2)
            layout[q1]=q2
        elif(len(splits)==1):
            time=splits[0]

    return layout,time




In [36]:
def retrieve_result(job,job_file_name,f_out,backend,qc,prev_qc,qBitArr_2,qBitArr,nQBit,p_id,ckt_name,time):
    if job is None:
        job_file=open(job_file_name,'r')
        print('p_id: '+p_id+' ckt_name: '+ckt_name)
        for line in job_file:
            print(line)
            line=line.split()
            print(line[0])
            print(line[1])
            if((line[0])==(p_id) and line[1]==ckt_name):
                print(line[2])
                job_id=line[2]
                break
        job_file.close()    
        retrieved_job = backend.retrieve_job(job_id)
    else:
        retrieved_job = job 
        
    counts=retrieved_job.result().get_counts(qc)

    print(counts)
    showResults(counts,nQBit,qBitArr_2)
            
    qbits = ''

    for i in range(len(qBitArr_2)):
        if(i!=len(qBitArr_2)-1):
            qbits = qbits + str(qBitArr_2[i]) + ':'
        else :
            qbits += str(qBitArr_2[i])
            # print('8')
    
    simResult = execute(qc, backend=sim_backend, shots=2).result() 
    simResult_ex_map = execute(prev_qc, backend=sim_backend, shots=2).result()

            

    expectedPattern_ex_map=getExpectedPattern(simResult_ex_map.get_counts(prev_qc),nQBit,qBitArr)
    expectedPattern=getExpectedPattern(simResult.get_counts(qc),nQBit,qBitArr_2)
    if(expectedPattern!=expectedPattern_ex_map):
        print('Mapping Error')
        exit(1)
        raise  Exception
    else:
        print('Mapping Okay')

    expectedResult, secondBest = getPerformanceMeasure(counts,nQBit,qBitArr_2,expectedPattern)
            
    f=open(f_out,'a')
    output=str(ckt_name)+' '+p_id+' '+str(time)+\
    ' '+qbits+' '+expectedPattern_ex_map+' '+expectedPattern+' '+str(expectedResult)+\
                       +' '+str(shots)+' '+ time+'\n'
    f.write(output)
            # print('11')
    print(output,'\n')

    f.close()




In [37]:
def monitor(job_file_name):
    job_file=open(job_file_name,'r')
    for line in job_file:
        print(line)
        line=line.split()
        
        print('p_id: ' + line[0]+'  '+'ckt name: '+line[1]+'\n')
        job_monitor(backend.retrieve_job(line[2]))
    job_file.close()
            


In [43]:
def run_map(nQBit,qBitArr,shots,job_file_name,fout,ckt_name,p_id):      
    config="config_m.txt"
    print("running mapper # "+str(p_id))
    prev_qc=QuantumCircuit().from_qasm_file(ckt_name)
    prev_qc.barrier(range(prev_qc.num_qubits))
    prev_qc.measure(range(prev_qc.num_qubits),range(prev_qc.num_qubits))
    layout, time= get_a_maped_qasm(ckt_name,p_id,config)
    qBitArr_2 = []
    for i in range(nQBit):
        qBitArr_2.append(layout[qBitArr[i]])
    print(qBitArr)
    print(qBitArr_2)
    
    out_ckt_name = 'out_'+p_id+'.qasm'
    qc=QuantumCircuit().from_qasm_file(out_ckt_name)
    qc.barrier(range(qc.num_qubits))
    qc.measure(range(qc.num_qubits),range(qc.num_qubits))
    job=execute(qc, backend=backend,shots=shots)
    
    job_monitor(job)
    retrieve_result(job,job_file_name,fout,backend,qc,prev_qc,qBitArr_2,
                qBitArr,nQBit,p_id,ckt_name,time)
    
    job_file=open(job_file_name,'a')
    job_file.write(str(p_id)+' '+ckt_name+' '+str(job.job_id())+'\n')
    job_file.close()
    print(job.job_id())
    
    
    
    

In [44]:
import datetime as dt
np.random.seed(77777)
ckt_map={'Adder':4,
         'BV4'  :4,
         'BV4_1111':4,
         'BV6':6,
         'BV6_111111':6 ,
         'BV8': 8,
         'Fredkin' :3,
         'HS2' : 2,
         'HS4' : 4,
         'HS6' : 6,
         'Or'  : 3,
         'Peres': 3,
         'Shor' :4,
         'Toffoli':3 
        }
qr=QuantumRegister(total_qubits);
cr=QuantumRegister(total_qubits);
name= input("give the circuit to execute: ")
if(name=='n'):
    name='Adder'
ckt_name=name+'.qasm'
nQBit=ckt_map[name]
qBitArr=range(nQBit)
shots=8192
job_file_name=input("give job file name: ")
if(job_file_name=='n'):
    job_file_name= 'job_file_n.txt'
elif(job_file_name=='t'):
    job_file_name = 'job_'+str(dt.date.today())+'.txt'
fout   = input("give output file name: ")
if(fout=='n'):
    fout= 'out_file_n.txt'
elif(fout=='t'):
    fout='out_'+str(dt.date.today())+'.txt'
    

p_id=(input("give p_id: "))


run_map(nQBit=nQBit,qBitArr=qBitArr,shots=shots,job_file_name=job_file_name,fout=fout,
                                ckt_name=ckt_name,p_id=str(p_id))
# monitor_job(job)






give the circuit to execute: Adder
give job file name: t
give output file name: t
give p_id: 1
running mapper # 1
range(0, 4)
[10, 11, 3, 4]
Job Status: job has successfully run
{'000000000000001': 28, '000100000010000': 31, '000100100001100': 1, '001001100000000': 1, '000000100010000': 73, '010000000000001': 1, '011000000000010': 1, '000111000001000': 3, '010100000100000': 1, '011000000000000': 6, '001100000011000': 1, '010000000011000': 6, '000000001000000': 30, '000100000001010': 4, '000000100000100': 24, '010001110000000': 1, '000000000000110': 3, '000100000001100': 4, '001001000000000': 2, '000110000010000': 29, '000001000000000': 35, '010100000010000': 2, '100110100000000': 1, '001110000000000': 1, '010100100011000': 1, '000010100000010': 3, '000010100011000': 10, '110000000100000': 1, '000000010010000': 3, '000000001001000': 1, '000000101000010': 1, '000010100110001': 1, '000110000001100': 1, '000101100001000': 1, '000110000101000': 1, '000011000011000': 2, '010010000011000': 2,

AttributeError: 'str' object has no attribute 'configuration'